We import the neccesary libraries

In [3]:
import pandas as pd
import requests
import sqlite3

Now we start by reading the data of US demographics csv file, and save it into the dataframe `us_d`

In [3]:
us_d = pd.read_csv('us-cities-demographics.csv', sep=';')
us_d.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
1,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
2,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
3,Hampton,Virginia,35.5,66214.0,70240.0,136454,19638.0,6204.0,2.48,VA,Black or African-American,70303
4,Lakewood,Colorado,37.7,76013.0,76576.0,152589,9988.0,14169.0,2.29,CO,Hispanic or Latino,33630


Following the instructions on the website, in this step the API url is used to extract the information about each city Air Quality Index (aqi) and also the concentration measurements for each mayor pollutant in two different lists. In this case we have the list of cities from us extracted from our previous Dataframe `us_d`. The resulting dictionaries contain dictionaries of each city and it's measurments.

A list of cities not available in the API is also created under `missing_cities`.

In [4]:
api_url = 'https://api.api-ninjas.com/v1/airquality?city='
dic = []
air_qi = []
missing_cities = []
for city in us_d['City'].unique():
    api_url = f'https://api.api-ninjas.com/v1/airquality?city={city}'
    response = requests.get(api_url, headers={'X-Api-Key': 'gBiipiwSiMtD3TpF+QIlYg==OAPqGhEzwXmgGZwS'})
    if response.status_code == requests.codes.ok:
        data = response.json()
        dic.append({'city':city})
        air_qi.append({'city':city})
        for key, value in data.items():
            if key != 'overall_aqi':
                dic.append({key:value.get('concentration')})
                air_qi.append({key:value.get('aqi')})
    else:
        print("Error:", response.status_code, f'Could not find city: {city}')
        missing_cities.append(city)


Error: 400 Could not find city: Augusta-Richmond County consolidated government
Error: 400 Could not find city: Urban Honolulu
Error: 400 Could not find city: Florence-Graham
Error: 400 Could not find city: Louisville/Jefferson County metro government


We used the list `dic` that contains all of the cities and it's measurments of major pollutants to create a dataframe with the name `us_aq_1` 

In [5]:
print(len(us_d['City'].unique()))
rows = []
for i in range(0, len(dic), 7):
    row = []
    for j in range(7):
        row.append(list(dic[i+j].values())[0])
    rows.append(row)
# Define the column names
columns = ['city', 'CO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10']
# Create the dataframe from the rows and columns
us_aq_1 = pd.DataFrame(rows, columns=columns)

567


We used the list `air_qi` that contains all of the cities and it's air quality indices of major pollutants to create a dataframe with the name `us_aqi`

In [6]:
rows = []
for i in range(0, len(air_qi), 7):
    row = []
    for j in range(7):
        row.append(list(air_qi[i+j].values())[0])
    rows.append(row)
# Define the column names
columns = ['city', 'CO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10']
# Create the dataframe from the rows and columns
us_aqi = pd.DataFrame(rows, columns=columns)
us_aqi.head(5)

,city,CO,NO2,O3,SO2,PM2.5,PM10
0,Newark,4,38,27,5,51,18
1,Peoria,4,26,4,0,18,6
2,O'Fallon,4,29,10,0,28,11
3,Hampton,3,7,51,4,29,9
4,Lakewood,3,18,37,0,10,5


This step ensures the cleaning of the data by removing the unnecesary columns and duplicated rows in the `us_d` dataframe

In [7]:
us_d.drop(columns=['Race', 'Count', 'Number of Veterans'], inplace= True, axis=1)
us_d.drop_duplicates(keep='first',inplace=True)

Here we create a SQLite database `air_quality_study.db` that has two tables. One table created from the `us_aqi` dataframe and the second table from `us_d` dataframe.

In [11]:
us_aqi.rename(columns={'PM2.5':'PM2_5'},inplace=True)

conn = sqlite3.connect('air_quality_study.db')
cur = conn.cursor()
us_aqi.to_sql('air_quality', conn, if_exists='replace', index=False)
us_d.to_sql('us_demographics', conn, if_exists='replace', index=False)
conn.close()

In this section we tried to check if the most populated cities also have the worst air quality.
To answer the question we first made a query into our `air_quality_study.db` to see which are the 10 most populated cities in the US usin the `us_demographics` table.
We then checked for the top 10 cities that had the worst air quality indeces for each major polutant and it's avarage. We saved the results of our queries in the list `results` 

In [29]:
conn = sqlite3.connect('air_quality_study.db')
cur = conn.cursor()
query_1 = 'SELECT "city" FROM "us_demographics" ORDER BY "total population" DESC LIMIT 10'
query_summary = 'SELECT "city" FROM (SELECT * FROM "air_quality") '
query_2 = 'SELECT "city" FROM "air_quality" ORDER BY "CO" LIMIT 10'
query_3 = 'SELECT "city" FROM "air_quality" ORDER BY "NO2" LIMIT 10'
query_4 = 'SELECT "city" FROM "air_quality" ORDER BY "O3" LIMIT 10'
query_5 = 'SELECT "city" FROM "air_quality" ORDER BY "SO2" LIMIT 10'
query_6 = 'SELECT "city" FROM "air_quality" ORDER BY "PM2_5" LIMIT 10'
query_7 = 'SELECT "city" FROM "air_quality" ORDER BY "PM10" LIMIT 10'
query_8 = 'SELECT "city" FROM (SELECT "city", ("CO"+"NO2"+"O3"+"SO2"+"PM2_5"+"PM10")/6 as "avarage" FROM "air_quality" ORDER BY "avarage" LIMIT 10)'
most_populated_cities = cur.execute(query_1).fetchall()
CO_most_contaminated = cur.execute(query_2).fetchall()
NO3_most_contaminated = cur.execute(query_3).fetchall()
O3_most_contaminated = cur.execute(query_4).fetchall()
SO2_most_contaminated = cur.execute(query_5).fetchall()
PM2_5_most_contaminated = cur.execute(query_6).fetchall()
PM10_most_contaminated = cur.execute(query_7).fetchall()
most_contaminated_avg = cur.execute(query_8).fetchall()
conn.close()
l1=[str(city[0]) for city in most_populated_cities]    
l1 = {'most_populated_cities':l1}
l2=[str(city[0]) for city in CO_most_contaminated]    
l2 = {'CO_most_contaminated':l2}
l3=[str(city[0]) for city in NO3_most_contaminated]    
l3 = {'NO3_most_contaminated':l3}
l4=[str(city[0]) for city in O3_most_contaminated]    
l4 = {'O3_most_contaminated':l4}
l5=[str(city[0]) for city in SO2_most_contaminated]    
l5 = {'SO2_most_contaminated':l5}
l6=[str(city[0]) for city in PM2_5_most_contaminated]    
l6 = {'PM2_5_most_contaminated':l6}
l7=[str(city[0]) for city in PM10_most_contaminated]    
l7 = {'PM10_most_contaminated':l7}
l8=[str(city[0]) for city in most_contaminated_avg]    
l8 = {'most_contaminated_avg':l8}
results = [l1,l2,l3,l4,l5,l6,l7,l8]


At last we created a Dataframe that contains all the results of our queries, which has the names of the cities with the most populated cities and the cities with the worst air quality indices for each major pollutant and the avarage overall. We ordered each column in alphabetic order independently.

In [32]:
# Create an empty dataframe with 10 rows
df = pd.DataFrame(index=range(10))
# Loop through the list and add each dictionary as a column to the dataframe
for d in results:
    # Get the key and value of the dictionary
    key = list(d.keys())[0]
    value = list(d.values())[0]
    # Add the column to the dataframe with the key as the name and the value as the data
    df[key] = value
# Looping through the columns and sort them by alphabetic order
for col in df.columns:
    # Sort the column and reset the index
    df[col] = df[col].sort_values().reset_index(drop=True)
df

,most_populated_cities,CO_most_contaminated,NO3_most_contaminated,O3_most_contaminated,SO2_most_contaminated,PM2_5_most_contaminated,PM10_most_contaminated,most_contaminated_avg
0,Chicago,Billings,Bayamón,Anaheim,Boulder,Bay,Bay,Bay
1,Dallas,Boulder,Caguas,Atlanta,Dothan,Bayonne,Boulder,Bayamón
2,Houston,Bryan,Carmichael,Brentwood,Flint,Brandon,Brandon,Caguas
3,Los Angeles,Carmichael,Carolina,Costa Mesa,Fort Smith,Buffalo,Buffalo,Guaynabo
4,New York,Dothan,German,Fullerton,Lakewood,Cheektowaga,Cambridge,Mayagüez
5,Philadelphia,Largo,Mayagüez,Jurupa Valley,Madison,Erie,Cheektowaga,Ponce
6,Phoenix,Mesa,New Britain,Kirkland,Mesa,Rochester,Erie,Rockford
7,San Antonio,San Clemente,Saint George,Moreno Valley,O'Fallon,Warwick,Rochester,San Clemente
8,San Diego,Shreveport,San Juan,Newport Beach,Peoria,Weston,Warwick,San Ramon
9,San Jose,Toledo,Weston,Santa Ana,Springfield,Yakima,Weston,Toledo


Then our last step is to check if the most populated cities had the worst air quality indices (aqi) by checking if this list of cities is within the list of the cities with the worst aqi for each major pollutant and it's avarage.
Even though we didn't find the top ten most populated cities didn't have the worst air quality, we recomend further analisys to check the relationship of population size with air quality. 

In [37]:
city_presence = df['most_populated_cities'].isin(df.drop('most_populated_cities', axis=1).values.ravel())
city_count = city_presence.sum()
print(f'number of the most populated cities that are within the top 10 most contaminated cities\ngiven different measurments (CO, NO2, O3, SO2, PM2_5, PM10, Avarage): {city_count}')

number of the most populated cities that are within the top 10 most contaminated cities
given different measurments (CO, NO2, O3, SO2, PM2_5, PM10, Avarage): 0
